# Create WaDE Uploader csv Files for Water Right and Time Series Water Use Data
- Last Updated: 07/22/2025
- Purpose: To create necessary processed WaDE Uploader csv files for water right information.

In [ ]:
# Needed Libraries / Modules
import sys
import os

## Custom Libraries
sys.path.append("../../5_CustomFunctions/MappingFunctions")
import CreateWaterSourcesFile
import CreateSitesFile
import CreateAllocationsAmounts_factsFile
import CreateSiteSpecificAmounts_factFile
import RemoveUnusedRecordsFile
import CreatePODSiteToPOUSiteRelationshipsFile
import JoinOverlayToSiteFile
import AddCountyHUC8HUC12File

## Input Data (make changes here)
 - create and add variables specific to this proejct here

In [ ]:
# ---- working directory ----
workingDirString = #"G:/Shared drives/WaDE Data/WaDE Data Folder/Texas/WaterAllocation_WaterUse"  # set working directory folder string here
os.chdir(workingDirString)
print(f'The working Directory is:', workingDirString)

# ---- Inputs ----
varST = #"AZ" # source organization or state abbreviation
varUUIDType = #"wr" # UUID data type abbreviation
mainInputFile = "RawinputData/Pwrwu_Main.zip" # use processed zip file

In [ ]:
# ---- Read in mainInputFile csv file ----
df = pd.read_csv(mainInputFile, compression='zip')
print(f"Total memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# ---- Optomize data types of mainInputFile csv file ----
def optimize_dataframe(df):
    for col in df.select_dtypes(include='int'):
        if (df[col] >= 0).all():
            df[col] = pd.to_numeric(df[col], downcast='unsigned')
        else:
            df[col] = pd.to_numeric(df[col], downcast='integer')

    for col in df.select_dtypes(include='float'):
        df[col] = pd.to_numeric(df[col], downcast='float')

    for col in df.select_dtypes(include='object'):
        num_unique_values = df[col].nunique()
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')

    return df

df = optimize_dataframe(df)
print(f"Total memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## Create CSV Input Files

In [ ]:
# ---- Method (methods.csv) ----
# Simple info, create by hand and save in ProcessedInputData folder.

In [ ]:
# ---- Variable (variables.csv) ----
# Simple info, create by hand and save in ProcessedInputData folder.

In [ ]:
# ---- Organization (organizations.csv) ----
# Simple info, create by hand and save in ProcessedInputData folder.

In [ ]:
# ---- Water Sources (watersources.csv) ----
print("Creating Water Sources input csv...")
print("############################################################################")
CreateWaterSourcesFile.CreateWaterSourcesInputFunction(workingDirString, varST, varUUIDType, df)

In [ ]:
# ---- Sites (sites.csv) ----
print("Creating Sites input csv...")
print("############################################################################")
CreateSitesFile.CreateSitesInputFunction(workingDirString, varST, varUUIDType, df)

In [ ]:
# ---- Allocations Amounts (waterallocations.csv) ----
print("Creating AllocationsAmounts_facts input csv...")
print("############################################################################")
CreateAllocationsAmounts_factsFile.CreateAllocationsAmounts_factsInputFunction(workingDirString, varST, varUUIDType, df)

In [ ]:
# ---- Site Specific Amounts (sitespecificamounts.csv) ----
print("Creating Site Specific Amounts input csv...")
print("############################################################################")
CreateSiteSpecificAmounts_factFile.CreateSiteSpecificAmounts_factsInputFunction(workingDirString, df)

In [ ]:
# ---- Remove unused records ----
print("Remove unused Water Sources and Sites records not found within waterallocations.csv...")
print("############################################################################")
RemoveUnusedRecordsFile.RemoveUnusedAllocationsAmountRecordsFileFunction(workingDirString)

In [ ]:
# ---- Create POD and POU joins (podsitetopousiterelationships.csv) ----
print("Creating Pod site -to- Pou site relationships input csv...")
print("############################################################################")
CreatePODSiteToPOUSiteRelationshipsFile.PODToPOUWaterRightRelationshipsFunction(workingDirString)

In [ ]:
# ---- Join OverlayUUIDs for Sites (Sites.csv update) ----
print("Joining OverlayUUIDs to Sites input csv if exists...")
print("############################################################################")
JoinOverlayToSiteFile.JoinOverlayToSiteFunction(workingDirString)

In [ ]:
# ---- Add missing County, HUC8, HUC12 information for Sites (Sites.csv update) ----
print("Add missing County, HUC8, HUC12 information if missing...")
print("############################################################################")
AddCountyHUC8HUC12File.AddCountyHUC8HUC12Function(workingDirString)